In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import zipfile

with zipfile.ZipFile('../input/dogs-vs-cats-redux-kernels-edition/test.zip') as existing_zip:
    existing_zip.extractall()
with zipfile.ZipFile('../input/dogs-vs-cats-redux-kernels-edition/train.zip') as existing_zip:
    existing_zip.extractall()

In [ ]:
ls ../input/dogs-vs-cats-redux-kernels-edition

In [ ]:
cd train

In [ ]:
mkdir cat

In [ ]:
mkdir dog

In [ ]:
import shutil
import glob
import os

def move_glob(dst_path, pathname, recursive=True):
    for p in glob.glob(pathname, recursive=recursive):
        shutil.move(p, dst_path)

move_glob('./dog', 'dog*.jpg')

move_glob('./cat', 'cat*.jpg')

In [ ]:
pip install openpyxl

In [ ]:
cd ../

In [ ]:
import torch # 基本モジュール
from torch.autograd import Variable # 自動微分用
import torch.nn as nn # ネットワーク構築用
import torch.optim as optim # 最適化関数
import torch.nn.functional as F # ネットワーク用の様々な関数
import torch.utils.data # データセット読み込み関連
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
import torchvision.models as models
import math
from torch.utils.data import Dataset
from torchvision import transforms

import matplotlib.pyplot as plt
import cloudpickle
import random
import cv2

import pandas as pd
from PIL import Image,ImageFilter
import numpy as np
import openpyxl as px

import sys
import time
import datetime
LABEL_IDX = 1
IMG_IDX = 0

In [ ]:
with open('./test.txt', mode='w') as f:
  f.write("name,label")
  for num in range(1,12501):
    f.write('\n')
    f.write("test/" + (str)(num) + ",0")
    
with open('./dog.txt', mode='w') as f:
  f.write("name,label")
  for num in range(1,12500):
    f.write('\n')
    f.write("dog/dog." + (str)(num) + ",0")
    
with open('./cat.txt', mode='w') as f:
  f.write("name,label")
  for num in range(1,12500):
    f.write('\n')
    f.write("cat/cat." + (str)(num) + ",1")
    
# 学習データのパス
TRAIN_PATH = [
    r"./cat.txt",
    r"./dog.txt",
    r"./test.txt"
]

# ミニバッチ
BATCH_SIZE = 100

In [ ]:
ls

In [ ]:
#データロード及びリサイズ
class uDataset(Dataset):

    def __init__(self, csv_file_path, root_dir, training=True):
        #pandasでcsvデータの読み出し
        self.image_dataframe = pd.read_csv(csv_file_path)
        self.root_dir = root_dir

        #学習or推論
        self.training = training

    def __len__(self):
        return len(self.image_dataframe)

    def __getitem__(self, idx):
        #dataframeから画像へのパスとラベルを読み出す
        label = self.image_dataframe.iat[idx, LABEL_IDX]

        img_name = os.path.join(self.root_dir, self.image_dataframe.iat[idx, IMG_IDX])

        path = img_name + '.jpg'
        val = np.mean(Image.open(path))

        # 選択したインデックスの画像の読み込み
        image = Image.open(path)

        # 画像への前処理
        if self.training == True:
            # 画像の切り出し範囲をランダムに決定
            if image.width > image.height:
                dff_leng = image.width - image.height
                shift = random.randint(0, dff_leng)
                image = image.crop((shift, 0, image.width - (dff_leng - shift), image.height))
            elif image.height > image.width:
                dff_leng = image.height - image.width
                shift = random.randint(0, dff_leng)
                image = image.crop((0, shift, image.width, image.height  - (dff_leng - shift)))
            else:
                image = image.crop((0, 0, image.width, image.height))

            p = random.randint(1, 100)
            if p > 50:
                image = image.transpose(Image.FLIP_LEFT_RIGHT)

        else:
            if image.width > image.height:
                dff_leng = image.width - image.height
                shift = dff_leng / 2
                image = image.crop((shift, 0, image.width - (dff_leng - shift), image.height))
            elif image.height > image.width:
                dff_leng = image.height - image.width
                shift = dff_leng / 2
                image = image.crop((0, shift, image.width, image.height  - (dff_leng - shift)))
            else:
                image = image.crop((0, 0, image.width, image.height))
            
        if image.width != 224 or image.height != 224:
            image = image.resize((224, 224),Image.BICUBIC)
            
        
        # numpy型への変換
        image = np.concatenate([image],2)
        imageNP = np.array(image).astype(np.float32)


        # 標準化を行う
        ave_1 = np.mean(imageNP[:,:,0])
        ave_2 = np.mean(imageNP[:,:,1])
        ave_3 = np.mean(imageNP[:,:,2])
        std_1 = np.std(imageNP[:,:,0])
        std_2 = np.std(imageNP[:,:,1])
        std_3 = np.std(imageNP[:,:,2])
        
        
        if std_1 > 0:
            imageNP[:,:,0] = (imageNP[:,:,0] - ave_1) / std_1
        if std_2 > 0:
            imageNP[:,:,1] = (imageNP[:,:,1] - ave_2) / std_2
        if std_3 > 0:
            imageNP[:,:,2] = (imageNP[:,:,2] - ave_3) / std_3

        image = transforms.ToTensor()(imageNP)


        return image, label
    
    def filepath(self, idx):
        img_name = os.path.join(self.root_dir, self.image_dataframe.iat[idx, IMG_IDX])
        return img_name

In [ ]:
#ネットワーク構造
class Block(nn.Module):
    def __init__(self, channel_in, channel_out):
        super().__init__()
        channel = channel_out // 4

        # 1x1 の畳み込み
        self.conv1 = nn.Conv2d(channel_in, channel,
                               kernel_size=(1, 1))
        self.bn1 = nn.BatchNorm2d(channel)
        self.relu1 = nn.ReLU()

        # 3x3 の畳み込み
        self.conv2 = nn.Conv2d(channel, channel,
                               kernel_size=(3, 3),
                               padding=1)
        self.bn2 = nn.BatchNorm2d(channel)
        self.relu2 = nn.ReLU()

        # 1x1 の畳み込み
        self.conv3 = nn.Conv2d(channel, channel_out,
                               kernel_size=(1, 1),
                               padding=0)
        self.bn3 = nn.BatchNorm2d(channel_out)

        # skip connection用のチャネル数調整        
        self.shortcut = self._shortcut(channel_in, channel_out)
        
        self.relu3 = nn.ReLU()

    def forward(self, x):
        #print("//////////////////////////////")
        #print(x.size())
        h = self.conv1(x)
        #print(h.shape)
        h = self.bn1(h)
        #print(h.shape)
        h = self.relu1(h)
        #print(h.shape)
        h = self.conv2(h)
        #print(h.shape)
        h = self.bn2(h)
        #print(h.shape)
        h = self.relu2(h)
        #print(h.shape)
        h = self.conv3(h)
        #print(h.shape)
        h = self.bn3(h)
        #print(h.shape)
        shortcut = self.shortcut(x)
        #print(h.shape)
        y = self.relu3(h + shortcut)  # skip connection
        #print(y.shape)
        return y

    def _shortcut(self, channel_in, channel_out):
        if channel_in != channel_out:
            return self._projection(channel_in, channel_out)
        else:
            return lambda x: x

    def _projection(self, channel_in, channel_out):
        return nn.Conv2d(channel_in, channel_out,
                         kernel_size=(1, 1),
                         padding=0)

class ResNet50(nn.Module):
    def __init__(self, output_dim):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 64,
                               kernel_size=(7, 7),
                               stride=(2, 2),
                               padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=(3, 3),
                                  stride=(2, 2),
                                  padding=1)

        # Block 1
        self.block0 = self._building_block(256, channel_in=64)
        
        self.block1 = nn.ModuleList([
            self._building_block(256) for _ in range(2)
        ])

        self.conv2 = nn.Conv2d(256, 512,
                               kernel_size=(1, 1),
                               stride=(2, 2))

        # Block 2
        self.block2 = nn.ModuleList([
            self._building_block(512) for _ in range(4)
        ])

        self.conv3 = nn.Conv2d(512, 1024,
                               kernel_size=(1, 1),
                               stride=(2, 2))

        # Block 3
        self.block3 = nn.ModuleList([
            self._building_block(1024) for _ in range(6)
        ])

        self.conv4 = nn.Conv2d(1024, 2048,
                               kernel_size=(1, 1),
                               stride=(2, 2))

        # Block 4
        self.block4 = nn.ModuleList([
            self._building_block(2048) for _ in range(3)
        ])

        self.avg_pool = GlobalAvgPool2d()  # TODO: GlobalAvgPool2d
        self.fc = nn.Linear(2048, 1000)
        self.out = nn.Linear(1000, output_dim)

    def forward(self, x):
        #print("?????????????????????????????????????")
        #print(x.size())
        h = self.conv1(x)
        #print(h.shape)
        h = self.bn1(h)
        #print(h.shape)
        h = self.relu1(h)
        #print(h.shape)
        h = self.pool1(h)
        #print(h.shape)
        h = self.block0(h)
        #print(h.shape)
        for block in self.block1:
            h = block(h)
        h = self.conv2(h)
        for block in self.block2:
            h = block(h)
        h = self.conv3(h)
        for block in self.block3:
            h = block(h)
        h = self.conv4(h)
        for block in self.block4:
            h = block(h)
        h = self.avg_pool(h)
        #print(h.shape)
        h = self.fc(h)
        #print(h.shape)
        h = torch.relu(h)
        #print(h.shape)
        h = self.out(h)
        #print(h.shape)
        y = torch.log_softmax(h, dim=-1)

        return y

    def _building_block(self,
                        channel_out,
                        channel_in=None):
        if channel_in is None:
            channel_in = channel_out
        return Block(channel_in, channel_out)

class GlobalAvgPool2d(nn.Module):
    def __init__(self,
                 device='cpu'):
        super().__init__()

    def forward(self, x):
        return F.avg_pool2d(x, kernel_size=x.size()[2:]).view(-1, x.size(1))

In [ ]:
def loadData():
    # データセットの作成
    dataset1 = uDataset(TRAIN_PATH[0], r"./train/")
    dataset2 = uDataset(TRAIN_PATH[1], r"./train/")
    dataset3 = uDataset(TRAIN_PATH[2], r"./")

    trainDataset = dataset1 + dataset2
    testDataset = dataset3

    # データの読み込み
    trainDataloader = torch.utils.data.DataLoader(trainDataset, batch_size=BATCH_SIZE, shuffle=True)
    testDataloader = torch.utils.data.DataLoader(testDataset, batch_size=256 , shuffle=True)

    return trainDataloader, testDataloader

# ハイパーパラメータの設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")                   # 学習に使用するデバイス
model = ResNet50(2).to(device)
criterion = nn.CrossEntropyLoss()                                                       # loss関数
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08)
trainDataloader, testDataloader = loadData()                                            # データの読み込み

    
def train(dataLoader, epoch):
    model.train()
    trainLoss = 0
    correct = 0

    for batchIdx, (image, label) in enumerate(dataLoader):
        image, label = image.to(device), label.to(device)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        trainLoss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(label.view_as(pred)).sum().item()
            
        loss.backward()
        optimizer.step()

        if batchIdx * BATCH_SIZE % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)], Loss: {:.6f}'.format(
                epoch, batchIdx * BATCH_SIZE, len(dataLoader.dataset),
                100. * batchIdx / len(dataLoader), loss.item()))
        
    return trainLoss, correct

def test(dataLoader):
    model.eval()
    testLoss = 0
    correct = 0
    with torch.no_grad():
        for(image, label) in dataLoader:
            
            #print()
            image, label = image.to(device), label.to(device)
            output = model(image)
            loss = criterion(output, label)
            testLoss += loss.item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(label.view_as(pred)).sum().item()

        testLoss /= len(dataLoader)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            testLoss, correct, len(dataLoader.dataset),
            100. * correct / len(dataLoader.dataset)))

    return testLoss, correct

def learning(model):

    #学習
    numEpochs = 30

    lossList = []
    accList = []

    # 学習の収束結果をtensorboardに出力
    writer = SummaryWriter(log_dir="./logs/sgd")
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    for epoch in range(1, numEpochs + 1):
        loss, acc = train(trainDataloader, epoch)

        #test(testDataloader)

        writer.add_scalar("loss/train", loss / len(trainDataloader), epoch)
        writer.add_scalar("accuracy/train", 100. * float(acc) / len(trainDataloader.dataset), epoch)

        lossList.append(loss / len(trainDataloader))
        accList.append(100. * float(acc) / len(trainDataloader.dataset))

        # 10エポック毎に結果を保存
        if epoch % 10 == 0:
            savePath = "./cnn_interlude_" + "{:05}".format(epoch)
            torch.save(model.state_dict(), savePath + ".weight")
            with open(savePath + ".pkl", 'wb') as f:
                cloudpickle.dump(model, f)
        
        scheduler.step()

    writer.close()
    
    # 最終結果を保存
    torch.save(model.state_dict(), './cnn_result.pkl')
    with open('cnn_result.pkl', 'wb') as f:
        cloudpickle.dump(model, f)

In [ ]:
ls

In [ ]:
       
def inference(folderIndex):

    
    book = px.Workbook()
    book.save('../' + 'submission.xlsx')
    wb = px.Workbook()
    ws = wb.active
    #Sheetの名前を設定
    ws.title = "Sheet_1"


    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
    

    # データセットの作成
    dataset = uDataset(TRAIN_PATH[folderIndex], r"./", training=False)
    dataLoader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False)

    with open('./cnn_result.pkl', 'rb') as f:
        model = cloudpickle.load(f)
        #shutil.copy2('./cnn_result.pkl', saveFolderPath + "cnn_result.pkl")
        #print(model)
        model.to(device)


        model.eval()
        a = 2
        ws.cell(row = 1, column = 2, value = "label")
        ws.cell(row = 1, column = 1, value = "id")

        with torch.no_grad():
            for idx, (image, label) in enumerate(dataLoader):
                image, label = image.to(device), label.to(device)
                start2 = time.time()
                output = model(image)
                #print(model(image))
                
                # softmaxで犬とする確率の確率を算出
                outputS = F.softmax(model(image), dim=1)
                
                #csvファイルに書き込み
                ws.cell(row = a, column = 2, value = float(outputS[0][0]))
                ws.cell(row = a, column = 1, value = a-1)
                a += 1
                print(a)
                
    wb.save('./' + 'submission.xlsx')

In [ ]:
if __name__ == '__main__':
    # 学習モデルのサイズ
    print('Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))
    learning(model)
    idx = 2
    inference(idx)
    change_csv = pd.read_excel('./' + 'submission.xlsx', sheet_name=0)
    change_csv.to_csv('./' + 'submission.csv', index=False)

In [ ]:
pwd